<a href="https://colab.research.google.com/github/milkyme/DSlab_practice/blob/main/Tensorflow01_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Neural Network
- The code below performs neural network which classifies binary category.
- The dataset is titanic data from external, where we want to classify whether one survives or not based on several numerical features.

In [131]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [132]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv')
# dftrain.shape (627, 10)
# dfeval.shape (264, 10)
dftrain.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


- 'sex', 'class', and 'alone' features are converted as numerical ones. 
- 'deck' is popped out because there is so many missing values.
- 'embark_town' is also popped out because it's relatively not connected to 'survived'.
- 'survived' is the feature we want to predict.

In [133]:
sex = dftrain.pop('sex')
dftrain['sex'] = (sex == 'male')*1.0              # 0 : female, 1 : male

dfclass = dftrain.pop('class')
dftrain['class'] = (dfclass == 'First')*1.0
dftrain['class'] = (dfclass == 'Second')*2.0
dftrain['class'] = (dfclass == 'Third')*3.0       # 1 : First, 2 : Second, 3 : Third

alone = dftrain.pop('alone')
dftrain['alone'] = (alone == 'y')*1.0             # 0 : n, 1 : y

deck = dftrain.pop('deck')                        # There are too many unknown data
embark_town = dftrain.pop('embark_town')          # Non-related with survived

survived = dftrain.pop('survived')                # It's label.

stats_df = dftrain.describe()
out_fields = ['mean','min','50%','max']
stats_df = stats_df.loc[out_fields]
stats_df

,age,n_siblings_spouses,parch,fare,sex,class,alone
mean,29.631308,0.545455,0.379585,34.385399,0.653907,1.631579,0.593301
min,0.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,28.000000,0.000000,0.000000,15.045800,1.000000,3.000000,1.000000
max,80.000000,8.000000,5.000000,512.329200,1.000000,3.000000,1.000000


In [134]:
features = list(dftrain.columns)
maximum = list(stats_df.loc[out_fields[3]])

for i in range(7):
  dftrain[features[i]] = dftrain[features[i]] / maximum[i]

stats_df = dftrain.describe()
out_fields = ['mean','min','50%','max']
stats_df = stats_df.loc[out_fields]
stats_df

,age,n_siblings_spouses,parch,fare,sex,class,alone
mean,0.370391,0.068182,0.075917,0.067116,0.653907,0.54386,0.593301
min,0.009375,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.350000,0.000000,0.000000,0.029367,1.000000,1.00000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [135]:
# Placeholder can't contain tensor object. It's for hatching the external variables.
x_data = dftrain
y_data = survived

In [172]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

X = tf.placeholder(tf.float32, shape=[None, 7])
Y = tf.placeholder(tf.float32)

w = tf.Variable(tf.random_uniform([7, 1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
hypothesis = tf.sigmoid(tf.matmul(X, w) + b)

cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*(tf.log(1-hypothesis)))

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

rate = tf.Variable(0.01)
optimizer = tf.train.GradientDescentOptimizer(rate)
train = optimizer.minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(1501):
  sess.run(train, feed_dict={X: x_data, Y: y_data})
  if step % 500 == 0:
    print("step : {0}, cost : {1}, accuracy : {2},".format(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}), sess.run(accuracy, feed_dict={X: x_data, Y: y_data})))
    print("w : {0}, b : {1}".format(sess.run(w), sess.run(b)))

step : 0, cost : 0.743588924407959, accuracy : 0.46251994371414185,
w : [[ 0.7924626 ]
 [-0.7097509 ]
 [ 0.15086468]
 [ 0.3462831 ]
 [ 0.42502728]
 [-0.4851378 ]
 [ 0.46134636]], b : [-0.4014945]
step : 500, cost : 0.6819787621498108, accuracy : 0.5894858837127686,
w : [[ 0.656323  ]
 [-0.690279  ]
 [ 0.15876691]
 [ 0.31847894]
 [ 0.18773167]
 [-0.46255404]
 [ 0.199224  ]], b : [-0.6531514]
step : 1000, cost : 0.6765748262405396, accuracy : 0.6045496463775635,
w : [[ 0.6159982 ]
 [-0.6640776 ]
 [ 0.17402098]
 [ 0.3065995 ]
 [ 0.14144006]
 [-0.33587363]
 [ 0.12511286]], b : [-0.66651833]
step : 1500, cost : 0.6734614968299866, accuracy : 0.6095709204673767,
w : [[ 0.58611804]
 [-0.6412139 ]
 [ 0.1857564 ]
 [ 0.2975143 ]
 [ 0.11349781]
 [-0.2326098 ]
 [ 0.07613003]], b : [-0.66540116]


- Neural network is comprised of several layers that have various nodes and activation function.
- By trying various combinations with manipulating the number of layers and nodes, I notice two things. First one is simply increasing the number of layers does not help to get a high accuracy(in some cases, it causes  even lower accuracy). Second one is increasing the number of nodes is of help to some degree, but it's not always in effect.(empirically, above 256 is meaningless for these dataset and model. Of course, these two things are valid only for this example.)
- I used some optimizers like SGD, RMSprop, and Adam, but there is no outstanding performed one.
- Categorical cross entropy is used as loss function because our goal is doing binary classifying.
- Finally, we can get about 70% accuracy.(It depends on the circumstances.)

In [176]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='sigmoid'),
    keras.layers.Dense(2, activation='sigmoid')
])

model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_data.values, y_data.values, epochs=5)

Train on 627 samples
Epoch 1/5
627/627 [==============================] - 0s 186us/sample - loss: 0.6603 - acc: 0.6124
Epoch 2/5
627/627 [==============================] - 0s 42us/sample - loss: 0.6398 - acc: 0.6124
Epoch 3/5
627/627 [==============================] - 0s 40us/sample - loss: 0.6234 - acc: 0.6667
Epoch 4/5
627/627 [==============================] - 0s 41us/sample - loss: 0.6018 - acc: 0.6858
Epoch 5/5
627/627 [==============================] - 0s 40us/sample - loss: 0.5874 - acc: 0.7241
